In [1]:
!ls "../../Activity-detection/automated_shopping_experience/activity_rec/frames"

In [43]:
!cat boundingBox.py

In [7]:
'''
    POSE ESTIMATION ANNOTATOR(NAIVE!)
    
    Modification of code at: https://stackoverflow.com/questions/5501192/how-to-display-picture-and-get-mouse-click-coordinate-on-it

    Instructions:
        1) Run this code
        2) Select an image from the -->>imageDir<<--
        3) Wait for the browser to complete loading and opening your image
        4) Draw the bounding boxes by FIRST SELECTING THE BOTTOM LEFT POINT and then top_right point
           Drag and adjust the bounding box if necessary
        5) Click on show me the code button
        6) Now start clicking on the image for adding the joint coordinates in order
        7) After all the points are marked(by clicking) switch to any other open window in your computer and switch back
            to the program gui. Now click any button to start labeling another image.(The program doesn't catch the key press
            otherwise, I can't figure out why)
        

'''

from tkinter import *
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk
import re
import os

from scrapeCoordinates import extractBoundingBoxCoords

full_coords = ''
# draw bounding box for getting coords
def getBoundingBoxCords(img_file):
    coords = extractBoundingBoxCoords(img_file)
    coords = ' '.join(coords.split(', '))
    print(coords)
    return coords

# add the bounding box coordinates to the label file
def addBoundingBoxCoords(img_file, f):
    #content_length = getNumLabeled(f)
    #print('[addBoundingBoxCoords]content_length', content_length)
    global content_length
    # -1 for null string
    if((content_length == 1) or (content_length == 42)): # 43 = 1 + (4 + 36) + 2('person2 ')
        full_coords = getBoundingBoxCords(img_file)
        f.write(full_coords)
        content_length += 4
        print('[addBoundingBoxCoords]:content_length', content_length)
        print('full_coords', full_coords)
    else:
        print("To label {}\n".format(label_dict[ ( (content_length-5) / 2) + 1 ]))
    '''elif((content_length -9) >= 36):
        print("First person is tagged. Enter any key to tag another image\n")
    ''' 
# initialise the label file
def initialiseLabelFile(label_file, f):
    global content_length
    content_length = getNumLabeled(f)
    
    if (os.stat(label_file).st_size == 0):
        f.write('person1 ')
        content_length += 1
    elif ((content_length-5) == MAX_LABEL_POINTS):
        f.write('person2 ')
        content_length += 1
    print('[initialiseLabelFile]:content_length', content_length)

# get the number of features labeled
def getNumLabeled(f):
    f.seek(0)
    contents = f.read()
    f.seek(0, 2)            
    content_length_local = len(contents.split(' '))

    return content_length_local - 1 # for the null string

# the gui code
def gui():
    root = Tk()
     #setting up a tkinter canvas with scrollbars
    frame = Frame(root, bd=2, relief=SUNKEN)
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)
    xscroll = Scrollbar(frame, orient=HORIZONTAL)
    xscroll.grid(row=1, column=0, sticky=E+W)
    yscroll = Scrollbar(frame)
    yscroll.grid(row=0, column=1, sticky=N+S)
    canvas = Canvas(frame, bd=0, xscrollcommand=xscroll.set, yscrollcommand=yscroll.set, height = root.winfo_screenheight())
    canvas.grid(row=0, column=0, sticky=N+S+E+W)
    xscroll.config(command=canvas.xview)
    yscroll.config(command=canvas.yview)
    frame.pack(fill=BOTH,expand=1)

    positionRight = int(root.winfo_screenwidth() - 600)
    #positionDown = int(root.winfo_screenheight()/2 - windowHeight/2)

    # Positions the window in the center of the page.
    root.geometry("+{}+{}".format(positionRight, 0))
 
    # function to be called when mouse is clicked
    def saveCoords(event):
        global content_length
        with open(label_file, 'a+') as f:   
            
            #content_length = getNumLabeled(f)

            if ((content_length-5) == MAX_LABEL_POINTS):
                print("Tagging first person complete. Start tagging second person or enter any key to tag another image\n")
                #f.write('\nperson2')
            elif((content_length-10) ==  (2 * MAX_LABEL_POINTS )):
                newImage()

            f.write('{} {} '.format(event.x, event.y))
            content_length += 2
            f.seek(0)
            #print(f.read())
                 
            labels_added = ((content_length-5) / 2) # for the 8 bounding box coords
            
            print("Last added {} cordinates\n".format(label_dict[labels_added]))
            
            #if ((content_length - 8) == MAX_LABEL_POINTS):
                # if MAX_LABEL_POINTS are labeled get new image to label after adding bounding box cordinates
                #getBoundingBoxCords()
                #newImage()
    def addNull(event):
        global content_length
        with open(label_file, 'a+') as f:   
            
            #content_length = getNumLabeled(f)

            if ((content_length-5) == MAX_LABEL_POINTS):
                print("Tagging first person complete. Start tagging second person or enter any key to tag another image\n")
                #f.write('\nperson2')
            elif((content_length-10) ==  (2 * MAX_LABEL_POINTS )):
                newImage()

            f.write('-1 -1 ')
            content_length += 2
            f.seek(0)
            #print(f.read())
                 
            labels_added = ((content_length-5) / 2) # for the 8 bounding box coords
            
            print("Last added {} cordinates\n".format(label_dict[labels_added]))
            
    # open new image to label
    def newImage(event=None):
        #print('enter')
        #os.system("start "+label_file)
        root.destroy()
        gui()
    
    #adding the image
    imageDir = "../../Activity-detection/automated_shopping_experience/activity_rec/frames"#r"B:\Projects\MLabs\Project\Annotation\Anntoation-pose-estimation\frames"
    img_file = askopenfilename(parent=root, initialdir=imageDir,title='Choose an image.')
    
    labelDir = "../../Activity-detection/automated_shopping_experience/activity_rec/persondata"
    pattern = '(.*)/(.*)'
    fileName = re.match(pattern, img_file).group(2)
    label_file = os.path.join(labelDir, fileName.split('.')[0] + '.txt')
    
    print(label_file)
    with open(label_file, 'a+') as f:
        # initialise the label file
        initialiseLabelFile(label_file, f)

        # add the bounding box coords for the image
        addBoundingBoxCoords(img_file, f)
    
    img = ImageTk.PhotoImage(Image.open(img_file))
    canvas.create_image(0,0,image=img,anchor="nw")
    canvas.config(scrollregion=canvas.bbox(ALL))
        
    
    
   
    # left mouseclick event
    canvas.bind("<Button 1>", saveCoords)
    # right mouseclick event
    #canvas.bind("<Button 3>",lambda x: root.destroy())  
    canvas.bind("<Button 3>", addNull)
    # press enter for next image
    canvas.focus_set()
    canvas.bind("<Key>", newImage)
    
    root.mainloop()
    
    return label_file
   

labels = ['nose', 'neck', 'Rsho', 'Relb', 'Rwri', 'Lsho', 'Lelb', 'Lwri', 'Rhip', 'Rkne', 'Rank', 'Lhip', 'Lkne', 'Lank', 'Leye', 'Reye', 'Lear', 'Rear']
label_index = [i for i in range(1, 19)]
        
label_dict = dict(zip(label_index, labels))
                 
if __name__ == "__main__":
    MAX_LABEL_POINTS = len(labels) * 2
    
    content_length = 0
    
    label_file = gui()
    '''
    labelDir = "../../Activity-detection/automated_shopping_experience/activity_rec/persondata"
    label_file = os.path.join(labelDir, '153' + '.txt')
    with open(label_file, 'a+') as f:
        # initialise the label file
        initialiseLabelFile(label_file, f)

        # add the bounding box coords for the image
        #addBoundingBoxCoords(img_file, f)
        print(getNumLabeled(f))
    '''  
    
    
    

../../Activity-detection/automated_shopping_experience/activity_rec/persondata/315.txt
[initialiseLabelFile]:content_length 1
[boundingBox.py] 10 171 336 725 
[scrapeCoordinates.py]:  10 171 336 725 
10 171 336 725 
[addBoundingBoxCoords]:content_length 5
full_coords 10 171 336 725 
Last added nose cordinates

Last added neck cordinates

Last added Rsho cordinates

Last added Relb cordinates

Last added Rwri cordinates

Last added Lsho cordinates

Last added Lelb cordinates

Last added Lwri cordinates

Last added Rhip cordinates

Last added Rkne cordinates

Last added Rank cordinates

Last added Lhip cordinates

Last added Lkne cordinates

Last added Lank cordinates

Last added Leye cordinates

Last added Reye cordinates

Last added Lear cordinates

Last added Rear cordinates

../../Activity-detection/automated_shopping_experience/activity_rec/persondata/316.txt
[initialiseLabelFile]:content_length 1
[boundingBox.py] 18 171 343 718 
[scrapeCoordinates.py]:  18 171 343 718 
18 171 343 7